In [6]:
from itertools import chain, combinations

def powerset(iterable):
    """
    powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)
    """
    xs = list(iterable)
    # note we return an iterator rather than a list
    return chain.from_iterable(combinations(xs,n) for n in range(len(xs)+1))

def is_closed_under_intersection(set_of_subsets_of_S, set_of_subsets_of_S_2):
    for s1, s2 in combinations(set_of_subsets_of_S, 2):
        if not(tuple(sorted(set(s1) & set(s2))) in set_of_subsets_of_S_2):
            return False
    return True

def is_closed_under_union(set_of_subsets_of_S, set_of_subsets_of_S_2):
    for s1, s2 in combinations(set_of_subsets_of_S, 2):
        if not(tuple(sorted(set(s1) | set(s2))) in set_of_subsets_of_S_2):
            return False
    return True

def all_closed_sets(list_of_sets_of_open_sets, S):
    return [tuple(tuple(sorted(set(S) - set(s))) for s in set_of_open_sets) for set_of_open_sets in list_of_sets_of_open_sets]

def all_open_kernel_operators(list_of_sets_of_open_sets, powset_of_S):
    list_of_open_kernel_operators = []
    for set_of_open_sets in list_of_sets_of_open_sets:
        open_kernel_operators = {}
        for s in powset_of_S:
            for os in sorted(set_of_open_sets, key = lambda s: len(s), reverse = True):
                if set(os) <= set(s):
                    open_kernel_operators[s] = os
                    break
        list_of_open_kernel_operators.append(open_kernel_operators)
    return list_of_open_kernel_operators

def all_closure_operators(list_of_sets_of_closed_sets, powset_of_S):
    list_of_closure_operators = []
    for set_of_closed_sets in list_of_sets_of_closed_sets:
        closure_operators = {}
        for s in powset_of_S:
            for cs in sorted(set_of_closed_sets, key = lambda s: len(s)):
                if set(s) <= set(cs):
                    closure_operators[s] = cs
                    break
        list_of_closure_operators.append(closure_operators)
    return list_of_closure_operators

def all_collections_of_neighborhood_systems(list_of_sets_of_open_sets, list_of_open_kernel_operators, powset_of_S, S):
    list_of_collections_of_neighborhood_systems = []
    for i in range(len(list_of_sets_of_open_sets)):
        collection_of_neighborhood_systems = {}
        for x in S:
            list_of_neighborhoods_of_x = []
            for v in powset_of_S:
                vo = list_of_open_kernel_operators[i][v]
                if x in vo:
                    list_of_neighborhoods_of_x.append(v)
            collection_of_neighborhood_systems[x] = tuple(list_of_neighborhoods_of_x)
        list_of_collections_of_neighborhood_systems.append(collection_of_neighborhood_systems)
    return list_of_collections_of_neighborhood_systems

def all_open_sets(powset_of_S, S):
    list_of_sets_of_open_sets = []
    powset_minus_empty_set_and_S = tuple(s for s in powset_of_S if s != () and s != S)
    for set_of_subsets_of_S in powerset(powset_minus_empty_set_and_S):
        if not(is_closed_under_intersection(set_of_subsets_of_S, set(set_of_subsets_of_S + ((), S)))):
            continue
        if not(is_closed_under_union(set_of_subsets_of_S, set(set_of_subsets_of_S + ((), S)))):
            continue
        set_of_subsets_of_S += ((), S)
        list_of_sets_of_open_sets.append(set_of_subsets_of_S)
    
    return list_of_sets_of_open_sets

def all_topologies(n):
    """
    S = {1, 2, ..., n}
    """
    powset_of_S = tuple(powerset(range(1, n + 1)))
    powset_of_S = tuple(tuple(sorted(t)) for t in powset_of_S)
    S = tuple(range(1, n + 1))
    list_of_sets_of_open_sets = all_open_sets(powset_of_S, S)
    list_of_sets_of_closed_sets = all_closed_sets(list_of_sets_of_open_sets, S)
    list_of_open_kernel_operators = all_open_kernel_operators(list_of_sets_of_open_sets, powset_of_S)
    list_of_closure_operators = all_closure_operators(list_of_sets_of_closed_sets, powset_of_S)
    list_of_collections_of_neighborhood_systems = all_collections_of_neighborhood_systems(list_of_sets_of_open_sets, list_of_open_kernel_operators, powset_of_S, S)
    return (list_of_sets_of_open_sets, list_of_sets_of_closed_sets, list_of_open_kernel_operators, list_of_closure_operators, list_of_collections_of_neighborhood_systems)

In [7]:
import pickle
t = all_topologies(3)
with open('topology_from_open_sets.dump', 'wb') as f:
    pickle.dump(t, f)

(l1, l2, l3, l4, l5) = t
print("All Topologies of {1, 2, 3}:")
print()
for i in range(len(l1)):
    if i == 0:
        print(str(i+1) + "st Topology:")
    if i == 1:
        print(str(i+1) + "nd Topology:")
    if i == 2:
        print(str(i+1) + "rd Topology:")
    if i >= 3:
        print(str(i+1) + "th Topology:")
    print("Open Sets: ")
    print(l1[i])
    print("Closed Sets: ")
    print(l2[i])
    print("Open Kernel Operator: ")
    print(l3[i])
    print("Closure Operator: ")
    print(l4[i])
    print("Neighborhood Systems: ")
    print(l5[i])
    print()

All Topologies of {1, 2, 3}:

1st Topology:
Open Sets: 
((), (1, 2, 3))
Closed Sets: 
((1, 2, 3), ())
Open Kernel Operator: 
{(): (), (1,): (), (2,): (), (3,): (), (1, 2): (), (1, 3): (), (2, 3): (), (1, 2, 3): (1, 2, 3)}
Closure Operator: 
{(): (), (1,): (1, 2, 3), (2,): (1, 2, 3), (3,): (1, 2, 3), (1, 2): (1, 2, 3), (1, 3): (1, 2, 3), (2, 3): (1, 2, 3), (1, 2, 3): (1, 2, 3)}
Neighborhood Systems: 
{1: ((1, 2, 3),), 2: ((1, 2, 3),), 3: ((1, 2, 3),)}

2nd Topology:
Open Sets: 
((1,), (), (1, 2, 3))
Closed Sets: 
((2, 3), (1, 2, 3), ())
Open Kernel Operator: 
{(): (), (1,): (1,), (2,): (), (3,): (), (1, 2): (1,), (1, 3): (1,), (2, 3): (), (1, 2, 3): (1, 2, 3)}
Closure Operator: 
{(): (), (1,): (1, 2, 3), (2,): (2, 3), (3,): (2, 3), (1, 2): (1, 2, 3), (1, 3): (1, 2, 3), (2, 3): (2, 3), (1, 2, 3): (1, 2, 3)}
Neighborhood Systems: 
{1: ((1,), (1, 2), (1, 3), (1, 2, 3)), 2: ((1, 2, 3),), 3: ((1, 2, 3),)}

3rd Topology:
Open Sets: 
((2,), (), (1, 2, 3))
Closed Sets: 
((1, 3), (1, 2, 3), ())


Closure Operator: 
{(): (), (1,): (1, 2), (2,): (2,), (3,): (2, 3), (1, 2): (1, 2), (1, 3): (1, 2, 3), (2, 3): (2, 3), (1, 2, 3): (1, 2, 3)}
Neighborhood Systems: 
{1: ((1,), (1, 2), (1, 3), (1, 2, 3)), 2: ((1, 2, 3),), 3: ((3,), (1, 3), (2, 3), (1, 2, 3))}

19th Topology:
Open Sets: 
((1,), (1, 2), (1, 3), (), (1, 2, 3))
Closed Sets: 
((2, 3), (3,), (2,), (1, 2, 3), ())
Open Kernel Operator: 
{(): (), (1,): (1,), (2,): (), (3,): (), (1, 2): (1, 2), (1, 3): (1, 3), (2, 3): (), (1, 2, 3): (1, 2, 3)}
Closure Operator: 
{(): (), (1,): (1, 2, 3), (2,): (2,), (3,): (3,), (1, 2): (1, 2, 3), (1, 3): (1, 2, 3), (2, 3): (2, 3), (1, 2, 3): (1, 2, 3)}
Neighborhood Systems: 
{1: ((1,), (1, 2), (1, 3), (1, 2, 3)), 2: ((1, 2), (1, 2, 3)), 3: ((1, 3), (1, 2, 3))}

20th Topology:
Open Sets: 
((2,), (3,), (2, 3), (), (1, 2, 3))
Closed Sets: 
((1, 3), (1, 2), (1,), (1, 2, 3), ())
Open Kernel Operator: 
{(): (), (1,): (), (2,): (2,), (3,): (3,), (1, 2): (2,), (1, 3): (3,), (2, 3): (2, 3), (1, 2, 3): (1, 